
# Análisis de Sentimiento de Reviews del iPhone (UPB)

**Autor:** Marcio Gonzales (UPB)  
**Descripción:** Notebook listo para clase práctica (MBA) que:
- Carga un CSV desde GitHub (**tu** repositorio).
- Detecta automáticamente la columna de texto y, si existe, la de rating.
- Ejecuta un **modelo multilingüe (1–5 estrellas)** para análisis de sentimiento.
- Muestra distribuciones, compara contra rating real (si hay), colapsa a **POS/NEU/NEG**.
- Genera **WordClouds** por clase (POS/NEU/NEG) y general.

> Ejecuta las celdas en orden. En Colab, la primera ejecución descargará el modelo (tarda 30–60s).


In [ ]:

# ===== 0) Dependencias =====
!pip install -q transformers torch pandas numpy matplotlib wordcloud nltk


In [ ]:

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import pipeline

# Display options
pd.set_option('display.max_colwidth', 200)


## 1) Cargar tu CSV desde GitHub (URL raw)

In [ ]:

# Reemplaza por la URL 'raw' de *tu* archivo en GitHub si cambia.
RAW_URL = "https://raw.githubusercontent.com/Marcio-byte/UPB/7df52ed2479cf04f54804c68dfb7cf6ba14f3759/iphone.csv"
# Alternativa (siempre el último de main, si el archivo está en 'main'):
# RAW_URL = "https://raw.githubusercontent.com/Marcio-byte/UPB/main/iphone.csv"

def read_csv_robusto(url):
    tries = [
        {"encoding": "utf-8", "sep": ","},
        {"encoding": "latin-1", "sep": ","},
        {"encoding": "utf-8", "sep": ";"},
        {"encoding": "latin-1", "sep": ";"},
    ]
    last_err = None
    for kw in tries:
        try:
            df = pd.read_csv(url, **kw)
            return df
        except Exception as e:
            last_err = e
    raise last_err

df = read_csv_robusto(RAW_URL)

print("Columnas originales:", list(df.columns))
display(df.head(3))
print("Filas totales:", len(df))


## 2) Detectar automáticamente columna de **texto** y (opcional) **rating**

In [ ]:

# Normalizar nombres de columnas (sin perder referencia al original)
original_cols = list(df.columns)
norm_cols = [re.sub(r'\s+', '_', str(c).strip().lower()) for c in original_cols]
norm_map = dict(zip(original_cols, norm_cols))
inv_norm_map = {v: k for k, v in norm_map.items()}
df.rename(columns=norm_map, inplace=True)

# Candidatos ampliados
candidatos_texto = {
    "review", "reviewtext", "text", "body", "content", "comentario",
    "opinion", "comentarios", "review_body", "review_text",
    "description", "summary", "title"
}
candidatos_rating = {
    "overall", "rating", "ratings", "stars", "estrellas", "score",
    "puntuacion", "star_rating", "ratingvalue", "rating_value"
}

# Detectar texto
col_texto = None
for c in df.columns:
    if c in candidatos_texto:
        col_texto = c
        break
if col_texto is None:
    # Heurística: mayor longitud media en columnas no numéricas
    texto_cands = [c for c in df.columns if df[c].dtype == 'object']
    if texto_cands:
        col_texto = max(texto_cands, key=lambda c: df[c].astype(str).str.len().mean())

# Detectar rating
col_rating = None
for c in df.columns:
    if c in candidatos_rating:
        col_rating = c
        break

print("Columna de texto detectada:", col_texto, " (original:", inv_norm_map.get(col_texto, col_texto), ")")
print("Columna de rating detectada:", col_rating, " (original:", inv_norm_map.get(col_rating, col_rating), ")")

# Limpieza suave
df = df.dropna(subset=[col_texto]).reset_index(drop=True)

# Parseo de rating (si existe)
import math
def parse_rating(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # '★★★★★' -> 5
    if set(s) <= set("★☆ "):
        return s.count("★") if "★" in s else np.nan
    # '5 out of 5'
    m = re.search(r'(\d+(?:\.\d+)?)', s)
    if m:
        try:
            return float(m.group(1))
        except:
            return np.nan
    try:
        return float(s)
    except:
        return np.nan

if col_rating is not None:
    df[col_rating + "_num"] = df[col_rating].apply(parse_rating)
    vals = df[col_rating + "_num"]
    if vals.notna().any():
        inside = vals.dropna().between(1, 5).mean()
        if inside >= 0.9:
            df[col_rating + "_num"] = vals.clip(1, 5)


## 3) Modelo de sentimiento (multilingüe 1–5 estrellas) y predicción

In [ ]:

classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Para clase: usar muestra para rapidez
N = min(200, len(df))
sample = df.sample(N, random_state=42).reset_index(drop=True)

textos = sample[col_texto].astype(str).tolist()
preds = classifier(textos)

sample["pred_label"] = [p["label"] for p in preds]   # "1 star"..."5 stars"
sample["pred_score"] = [p["score"] for p in preds]

display(sample[[col_texto, "pred_label", "pred_score"]].head(10))


## 4) Distribución de sentimiento predicho

In [ ]:

plt.figure()
sample["pred_label"].value_counts().sort_index().plot(kind="bar", title="Distribución de sentimiento (predicho)")
plt.xlabel("Etiqueta")
plt.ylabel("Cantidad")
plt.show()


## 5) Comparación con rating real (si existe)

In [ ]:

import numpy as np
def label_to_num(lbl):
    m = re.match(r"(\d+)", str(lbl))
    return int(m.group(1)) if m else np.nan

if col_rating is not None:
    sample["pred_num"] = sample["pred_label"].apply(label_to_num)
    base_rating_col = col_rating + "_num" if (col_rating + "_num") in sample.columns else col_rating
    if base_rating_col not in sample.columns:
        sample[base_rating_col] = sample[base_rating_col].apply(parse_rating)
    print("\nTabla de contingencia (rating real vs predicción):")
    comp = sample.groupby([base_rating_col, "pred_num"]).size().unstack(fill_value=0)
    display(comp)

    # Discrepancias fuertes
    malos = sample[
        (sample[base_rating_col].round(0).between(4, 5)) &
        (sample["pred_num"].isin([1, 2]))
    ][[col_texto, base_rating_col, "pred_label"]].head(5)

    print("\nEjemplos donde usuario puso ≥4★ pero el modelo predijo 1–2★:")
    for _, row in malos.iterrows():
        print(f"- ({row[base_rating_col]}★ vs {row['pred_label']}) {row[col_texto][:200]}...")
else:
    print("No se detectó columna de rating en el CSV. Se omite la comparación.")


## 6) Colapsar a POS / NEU / NEG

In [ ]:

def to_sentiment_3(label):
    s = str(label).strip().lower()
    m = re.match(r"(\d+)", s)
    if m:
        n = int(m.group(1))
        if n <= 2:
            return "NEG"
        elif n == 3:
            return "NEU"
        else:
            return "POS"
    if "neg" in s: return "NEG"
    if "neu" in s: return "NEU"
    if "pos" in s or "star" in s: return "POS"
    return "NEU"

sample["pred_sent_3"] = sample["pred_label"].apply(to_sentiment_3)

ax = sample["pred_sent_3"].value_counts().reindex(["NEG","NEU","POS"]).plot(
    kind="bar", title="Distribución de sentimiento (POS / NEU / NEG)"
)
ax.set_xlabel("Clase"); ax.set_ylabel("Cantidad")
plt.show()


## 7) WordClouds por clase (POS / NEU / NEG) y general

In [ ]:

# Dependencias y stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

stop_en = set(stopwords.words('english'))
stop_es = set(stopwords.words('spanish'))
stop_extra = {
    "iphone", "iphones", "apple", "ios", "pro", "max", "phone", "phones",
    "cell", "celular", "celulares", "telefono", "teléfono",
    "review", "reviews", "producto", "producto.", "producto,", "producto;",
    "muy", "mas", "más", "etc", "rt"
}
stop_all = set(STOPWORDS) | stop_en | stop_es | stop_extra

def limpiar_texto(s):
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)  # URLs
    s = re.sub(r"[@#]\w+", " ", s)            # menciones/hashtags
    s = re.sub(r"[^a-záéíóúñü\s]", " ", s)    # signos/números
    s = re.sub(r"\s+", " ", s).strip()
    return s

def generar_wordcloud(texto_concatenado, titulo="WordCloud"):
    if not texto_concatenado.strip():
        print(f"[{titulo}] No hay texto suficiente para generar la nube.")
        return
    wc = WordCloud(width=1200, height=600, background_color="white",
                   stopwords=stop_all, max_words=200).generate(texto_concatenado)
    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(titulo)
    plt.show()

df_wc = sample.dropna(subset=[col_texto, "pred_sent_3"]).copy()
df_wc["texto_limpio"] = df_wc[col_texto].apply(limpiar_texto)

texto_pos = " ".join(df_wc.loc[df_wc["pred_sent_3"]=="POS", "texto_limpio"].tolist())
texto_neu = " ".join(df_wc.loc[df_wc["pred_sent_3"]=="NEU", "texto_limpio"].tolist())
texto_neg = " ".join(df_wc.loc[df_wc["pred_sent_3"]=="NEG", "texto_limpio"].tolist())

generar_wordcloud(texto_pos, "Palabras clave (POS)")
generar_wordcloud(texto_neu, "Palabras clave (NEU)")
generar_wordcloud(texto_neg, "Palabras clave (NEG)")

# WordCloud general
texto_all = " ".join(df_wc["texto_limpio"].tolist())
generar_wordcloud(texto_all, "Palabras clave (TODAS)")



---

### Notas para clase
- Si la ejecución es lenta por la descarga del modelo, corre esta sección **antes** de empezar.
- Ajusta `N` (muestra) en la celda del modelo si necesitas más/menos rapidez.
- Si cambias el nombre o ubicación del CSV, actualiza `RAW_URL`.
- Para comparar con rating real, asegúrate de que el CSV tenga una columna de estrellas reconocible.
